In [1]:
#  cd OneDrive\INSPER\PADS\TCC\FINAL\TABELA_ERRO_PREVISÃO\

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import math

In [4]:
import itertools
from functools import reduce

In [5]:
from matplotlib import pyplot as plt
import matplotlib.pyplot as ply

In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

In [7]:
#import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interactive,interactive_output
from ipywidgets import Layout, Button, Label
from ipywidgets import GridspecLayout,HBox, VBox, Box

In [8]:
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots

In [9]:
def df_color(df,cmap,low,high): 
    dff=df.style.background_gradient(cmap=cmap, low=low, high=high)
    return(dff)

In [10]:
regressao=['knn','gb','mlr','rf']
regressao_mlr=['knn','gb','rf']

In [11]:
# ERRO

In [12]:
url='https://raw.githubusercontent.com/LeonardoGiovanelli/TCC_LCA/main/erro2/df_erro_EQM_indici_data_regressao_tsw_5.csv'
erro_adjested_tidy=pd.read_csv(url)
erro_adjested_tidy=erro_adjested_tidy.set_index(['Indice','Nd','Regressao','tsw'])
erro_adjested_tidy=erro_adjested_tidy.round(2)
erro_adjested_tidy.rename(columns={'Mes':"Mês"},inplace=True)

In [13]:
indices=list(erro_adjested_tidy.index.levels[0])

In [14]:
teste=erro_adjested_tidy

teste_0=teste[np.in1d(teste.index.get_level_values(1), [0])]
teste_0=teste_0.groupby(['Indice','Regressao']).mean()
teste_0f=teste_0.reset_index().pivot(index='Indice', columns='Regressao', values='Real-Previsto(%)').round(0)


teste_1=teste[np.in1d(teste.index.get_level_values(1), [1])]
teste_1=teste_1.groupby(['Indice','Regressao']).mean()
teste_1f=teste_1.reset_index().pivot(index='Indice', columns='Regressao', values='Real-Previsto(%)').round(0)

In [15]:
def plotly_real_prev_erro(indice,regressao):
 
 ############ get data 
    colors=[["black","grey"],["maroon","firebrick"]]
    if regressao != 'knn':
                
        fig = make_subplots(rows=2, cols=2)   
        min=[]
        max=[]
        for n_nd in range(0,2):
            for n_tsw in range(0,2):
                
                r=add_data(erro_adjested_tidy.loc[indice,n_nd,regressao,n_tsw])              
                
                min.append(np.min(r['Previsto']-r['Erro']))
                max.append(np.max(r['Previsto']+r['Erro']))                 

                fig.add_trace(go.Scatter(
                line=dict(color=colors[n_nd][n_tsw]),
                x=r['data'],
                y=r['Previsto'],
                 error_y=dict(
                type='data',
                symmetric=True,
                array=r['Erro']),
                 name="nd="+str(n_nd)+" tsw="+str(n_tsw)),
                            row=n_tsw+1,
                              col=n_nd+1)

                fig.add_trace(go.Scatter(
                    line=dict(color="blue"),
                    x=r['data'],
                    y=r['Real'],
                   name="Real"),
                             row=n_tsw+1,
                              col=n_nd+1)

            
        fig.update_xaxes(dtick="M1",    tickformat="%b\n%Y")
        fig.update_layout(title={'text':indice,'x':0.5,'xanchor': 'center'})
        fig.show()
        
    else: 
        fig = make_subplots(rows=1, cols=2)
        colors=colors[0]         
        min=[]
        max=[]
        for n_nd in range(0,2):
                
            r=add_data(erro_adjested_tidy.loc[indice,n_nd,regressao,0])              

            min.append(np.min(r['Previsto']-r['Erro']))
            max.append(np.max(r['Previsto']+r['Erro']))

            fig.add_trace(go.Scatter(
            x=r['data'],
            y=r['Previsto'],
            line=dict(color=colors[n_nd]),
             error_y=dict(
            type='data',
            symmetric=True,
            array=r['Erro']),
             name="nd="+str(n_nd)+" tsw="+str(0)),
                         row=1,col=n_nd+1)
            
            fig.add_trace(go.Scatter(
            line=dict(color="blue"),
            x=r['data'],
            y=r['Real'],
            name="Real"),
                     row=1,
                      col=n_nd+1)
            
        fig.show()

In [16]:
#previsao

In [17]:
url='https://raw.githubusercontent.com/LeonardoGiovanelli/TCC_LCA/main/previsao2/df_PREVISAO_indici_data_regressao_tsw.csv'
previsao=pd.read_csv(url).iloc[:,1:]
previsao=previsao.round(2)
previsao=previsao.set_index(['Indice','Nd','Regressao','tsw'])
previsao=previsao.rename(columns={"Eroo": "Erro"})
#previsao.head()

In [18]:
#previsao.loc['IBC_BR',:,regressao_mlr]

In [19]:
#previsao.loc['EPU',:,regressao_mlr].groupby(['Ano','Mês']).apply(lambda x: np.average(x.Previsão, weights=x['EQM(%)']))

In [20]:
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / (w.sum())

In [21]:
from plotnine import *
from plotnine.data import *

from plotnine import ggplot, aes, geom_line,geom_smooth
%matplotlib inline

In [22]:
def add_data(r):
    r['Dia']="1"
    r["Ano"]=r["Ano"].apply(lambda x: math.trunc(x))
    r["Mês"]=r["Mês"].apply(lambda x: math.trunc(x))
    r['data']=pd.to_datetime(r["Ano"].astype(str) + "/" + r["Mês"].astype(str) + "/" + r["Dia"])
    return(r)

In [23]:
def plotly_real_prev(indice,regressao):
 
    fig = go.Figure()
    colors=[["black","grey"],["maroon","firebrick"]]
    if regressao != 'knn':      
          
          
        min=[]
        max=[]
        for n_nd in range(0,2):
            for n_tsw in range(0,2):
                
                r_aux=add_data(previsao.loc[indice,n_nd,regressao,n_tsw])              
                
                min.append(np.min(r_aux['Previsão']-r_aux['Erro']))
                max.append(np.max(r_aux['Previsão']+r_aux['Erro']))
                
                fig.add_trace(go.Bar(
                #line=dict(color=colors[n_nd][n_tsw]),
                marker={'color': colors[n_nd][n_tsw]},
               #width=[3]*len(r_aux),
                x=r_aux['data'],
                y=r_aux['Previsão'],
                 error_y=dict(
                type='data',
                symmetric=True,
                array=r_aux['Erro']),
                 name="nd="+str(n_nd)+" tsw="+str(n_tsw)))
    else: 
        colors=colors[0]
          
        min=[]
        max=[]
        for n_nd in range(0,2):
                
            r_aux=add_data(previsao.loc[indice,n_nd,regressao,0])              

            min.append(np.min(r_aux['Previsão']-r_aux['Erro']))
            max.append(np.max(r_aux['Previsão']+r_aux['Erro']))

            fig.add_trace(go.Bar(
            #line=dict(color=colors[n_nd][n_tsw]),
            marker={'color': colors[n_nd]},
           #width=[3]*len(r_aux),
            x=r_aux['data'],
            y=r_aux['Previsão'],
             error_y=dict(
            type='data',
            symmetric=True,
            array=r_aux['Erro']),
             name="nd="+str(n_nd)+" tsw="+str(0)))

#################
    fig.update_layout(yaxis_range=[np.min(min),np.max(max)])
    fig.update_layout(title={'text':indice,'x':0.5,'xanchor': 'center'})
    fig.update_xaxes(dtick="M1",    tickformat="%b\n%Y")
    fig.show()

In [24]:
####ipywidgets

In [25]:
indices=ipywidgets.Dropdown(options=indices,  value=indices[0],  
                            description='Índice: ',
                           layout={'width': 'max-content'})

nd=ipywidgets.Dropdown(options=[0,1],  value=0, 
                       description='nd:',
                      layout={'width': 'max-content'})

regressao=ipywidgets.Dropdown(options=regressao,  value=regressao[0],  
                              description='Regressão:',
                             layout={'width': 'max-content'})

tsw=ipywidgets.Dropdown(options=[0,1],  value=0,  
                        description='tsw:',
                       layout={'width': 'max-content'})


info=ipywidgets.HBox([indices,nd,regressao,tsw])

In [26]:
from IPython.display import display_html 
    
out1 = ipywidgets.Output()
out2 = ipywidgets.Output()
out3= ipywidgets.Output()

######## OUT1##################
with out1:
    i=0.1
    f=.4
    d1=df_color(teste_0f.astype(int).transpose(),"plasma",i,f).\
    set_table_attributes("style='display:inline'").set_caption('Português (nd=0)')
      
    d2=df_color(teste_1f.astype(int).transpose(),"plasma",i,f).\
    set_table_attributes("style='display:inline'").set_caption('Inglês (nd=1)')
    
    display_html(d1._repr_html_()+d2._repr_html_(), raw=True)        
###### OUT 2#####################    

g2=ipywidgets.interactive_output(plotly_real_prev_erro,  
    {'indice':indices,'regressao':regressao})

out2=ipywidgets.VBox([ipywidgets.HBox([indices,regressao]),g2])

####   OUT3   #####################    
g3=ipywidgets.interactive_output(plotly_real_prev,  
    {'indice':indices,'regressao':regressao})

out3=ipywidgets.VBox([ipywidgets.HBox([indices,regressao]),g3])

    
    
##############################    
tab = ipywidgets.Tab(children = [out1, out2,out3])
tab.set_title(0, 'Erro médio (%)')
tab.set_title(1, 'Erro plotly')
tab.set_title(2, 'Previsão')

display(tab)